In [4]:
import networkx as nx
import sqlite3
import pandas as pd

conn = sqlite3.connect('/Users/Admin/Documents/Projects/Kairos/main/data/GTFS.db')

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,stop_code,stop_street,stop_city,stop_region,stop_postcode,stop_country,location_type,parent_station,stop_timezone
0,18848,Clínicas,None,-23.55419,-46.670723,None,None,None,None,None,None,None,None,None,None,None


In [56]:
MG = nx.MultiDiGraph() # initialise Graph

In [57]:
## Add Nodes

In [58]:
stops = pd.read_sql_query('SELECT * FROM gtfs_stops', conn)
stops.head(1)

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,stop_code,stop_street,stop_city,stop_region,stop_postcode,stop_country,location_type,parent_station,stop_timezone
0,18848,Clínicas,None,-23.55419,-46.670723,None,None,None,None,None,None,None,None,None,None,None


In [59]:
for i, row in stops.iterrows():
    MG.add_node(row['stop_id'], location={'lat': row['stop_lat'], 'long': row['stop_lon']})

## Add Edges

In [61]:
stops_time = pd.read_sql_query('SELECT * FROM gtfs_stop_times', conn)
stops_time['arrival_time'] = stops_time['arrival_time'].apply(unicode_to_seconds)
stops_time['departure_time'] = stops_time['departure_time'].apply(unicode_to_seconds)
stops_time.head(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,13200,13200,301708,1,None,None,None,None,None,13200,13200


In [62]:
def unicode_to_seconds(a):
    a = map(int, a.split(':'))
    return a[0] * 3600 + a[1] * 60 + a[2]

In [83]:
for i, row in stops_time.iterrows():
    
    try:
        partida, chegada = stops_time.iloc[i], stops_time.iloc[i + 1]
    except:
        break
    if chegada['trip_id'] == partida['trip_id']:
        MG.add_edge(partida['stop_id'], chegada['stop_id'], 
                    attr = {'tempo': chegada['departure_time']-partida['arrival_time'],
                      'trip_id': chegada['trip_id']}
        )

{}